In [1]:
import numpy as np
from rdkit.Chem import AllChem
from rdkit.Chem import rdchem
#from pahelix.utils.compound_tools import *
#from pahelix.datasets.inmemory_dataset import InMemoryDataset
from rdkit import rdBase
rdBase.rdkitVersion

'2020.09.1'

In [9]:
import pickle
with open("filename.pickle","rb") as f:
    a=pickle.load(f)
a["pos"]

[array([[-3.4902122e+00,  4.1052926e-01, -2.5672016e-03],
        [-2.9398146e+00, -8.7031245e-01, -3.5819612e-02],
        [-1.5487598e+00, -1.0506483e+00, -3.5261244e-02],
        [-9.7747803e-01, -2.3299501e+00, -6.8385854e-02],
        [ 4.1026965e-01, -2.4931827e+00, -6.7366831e-02],
        [ 1.2627536e+00, -1.3813478e+00, -3.3194207e-02],
        [ 2.6576860e+00, -1.5287094e+00, -3.1703118e-02],
        [ 3.4902132e+00, -4.1053009e-01,  2.5690880e-03],
        [ 2.9398139e+00,  8.7031144e-01,  3.5824206e-02],
        [ 1.5487597e+00,  1.0506505e+00,  3.5260666e-02],
        [ 9.7747821e-01,  2.3299541e+00,  6.8391450e-02],
        [-4.1027081e-01,  2.4931827e+00,  6.7373693e-02],
        [-1.2627522e+00,  1.3813462e+00,  3.3193815e-02],
        [-2.6576843e+00,  1.5287081e+00,  3.1702351e-02],
        [-7.0365632e-01,  8.2766078e-02, -5.2152580e-04],
        [ 7.0365864e-01, -8.2766049e-02,  5.1363662e-04]], dtype=float32),
 array([[ 2.2178767e+00,  1.5128166e-03,  1.1190279e-01

In [30]:
mol=esol_pickle[sample_smiles][0]

In [21]:
type(atom_poses)

list

In [31]:
if len(mol.GetAtoms()) <= 400:
    mol, atom_poses = Compound3DKit.get_MMFF_atom_poses(mol, numConfs=10)
else:
    atom_poses = Compound3DKit.get_2d_atom_poses(mol)
atom_poses = Compound3DKit.get_atom_poses(mol, mol.GetConformer())
data_dic_geo=mol_to_geognn_graph_data(mol, atom_poses, dir_type='HT')

In [ ]:
dataset = InMemoryDataset(data_list)

In [34]:
data_dic_geo["smiles"]

KeyError: 'smiles'

In [36]:
data_dic_geo.keys()

dict_keys(['atomic_num', 'chiral_tag', 'degree', 'explicit_valence', 'formal_charge', 'hybridization', 'implicit_valence', 'is_aromatic', 'total_numHs', 'mass', 'bond_dir', 'bond_type', 'is_in_ring', 'edges', 'morgan_fp', 'maccs_fp', 'daylight_fg_counts', 'atom_pos', 'bond_length', 'BondAngleGraph_edges', 'bond_angle'])

In [45]:
for key in data_dic.keys():
    if key not in new_dataset[0].keys():
        print(key)

atomic_num.seq_len
atomic_num.singular
chiral_tag.seq_len
chiral_tag.singular
degree.seq_len
degree.singular
explicit_valence.seq_len
explicit_valence.singular
formal_charge.seq_len
formal_charge.singular
hybridization.seq_len
hybridization.singular
implicit_valence.seq_len
implicit_valence.singular
is_aromatic.seq_len
is_aromatic.singular
total_numHs.seq_len
total_numHs.singular
mass.seq_len
mass.singular
bond_dir.seq_len
bond_dir.singular
bond_type.seq_len
bond_type.singular
is_in_ring.seq_len
is_in_ring.singular
edges.seq_len
edges.singular
morgan_fp.seq_len
morgan_fp.singular
maccs_fp.seq_len
maccs_fp.singular
daylight_fg_counts.seq_len
daylight_fg_counts.singular
atom_pos.seq_len
atom_pos.singular
bond_length.seq_len
bond_length.singular
BondAngleGraph_edges.seq_len
BondAngleGraph_edges.singular
bond_angle.seq_len
bond_angle.singular
label
label.seq_len
label.singular
smiles
smiles.seq_len
smiles.singular


In [ ]:
dataset = InMemoryDataset(data_list)

In [3]:
f=open("cached_data/bbbp/bbbp.pkl","rb")
esol_pickle=pickle.load(f)

In [5]:
bbbp_geo_smiles=list(esol_pickle.keys())

In [11]:
import pandas as pd
bbbp_smiles=pd.read_csv("chemrl_downstream_datasets/bbbp/raw/BBBP.csv")["smiles"]

In [13]:
count=0
for smiles in bbbp_smiles:
    if smiles in bbbp_geo_smiles:
        count+=1
count

1928

In [ ]:
esol_pickle

In [7]:
sample_smiles="OCC3OC(OCC2OC(OC(C#N)c1ccccc1)C(O)C(O)C2O)C(O)C(O)C3O "

In [53]:
data_dic["atom_pos"].shape

(14991, 3)

In [37]:
data=np.load("cached_data/esol/part-000000.npz")
data_dic=dict(data)
len(data_dic["smiles"])
for smiles in data_dic["smiles"]:
    mol=esol_pickle[smiles][0]
    atom_poses = Compound3DKit.get_atom_poses(mol, mol.GetConformer())
    new_mol_dic=mol_to_geognn_graph_data(mol, atom_poses, dir_type='HT')
    new_dataset.append(new_mol_dic)
    
    

In [40]:
new_dataset = InMemoryDataset(new_dataset)

In [43]:
new_dataset[0].keys()

dict_keys(['atomic_num', 'chiral_tag', 'degree', 'explicit_valence', 'formal_charge', 'hybridization', 'implicit_valence', 'is_aromatic', 'total_numHs', 'mass', 'bond_dir', 'bond_type', 'is_in_ring', 'edges', 'morgan_fp', 'maccs_fp', 'daylight_fg_counts', 'atom_pos', 'bond_length', 'BondAngleGraph_edges', 'bond_angle'])

In [28]:
data=np.load("cached_data/esol/part-000000.npz")

In [29]:
data_dic=dict(data)
len(data_dic["smiles"])

1128

In [21]:
data_dic.keys()

dict_keys(['atomic_num', 'atomic_num.seq_len', 'atomic_num.singular', 'chiral_tag', 'chiral_tag.seq_len', 'chiral_tag.singular', 'degree', 'degree.seq_len', 'degree.singular', 'explicit_valence', 'explicit_valence.seq_len', 'explicit_valence.singular', 'formal_charge', 'formal_charge.seq_len', 'formal_charge.singular', 'hybridization', 'hybridization.seq_len', 'hybridization.singular', 'implicit_valence', 'implicit_valence.seq_len', 'implicit_valence.singular', 'is_aromatic', 'is_aromatic.seq_len', 'is_aromatic.singular', 'total_numHs', 'total_numHs.seq_len', 'total_numHs.singular', 'mass', 'mass.seq_len', 'mass.singular', 'bond_dir', 'bond_dir.seq_len', 'bond_dir.singular', 'bond_type', 'bond_type.seq_len', 'bond_type.singular', 'is_in_ring', 'is_in_ring.seq_len', 'is_in_ring.singular', 'edges', 'edges.seq_len', 'edges.singular', 'morgan_fp', 'morgan_fp.seq_len', 'morgan_fp.singular', 'maccs_fp', 'maccs_fp.seq_len', 'maccs_fp.singular', 'daylight_fg_counts', 'daylight_fg_counts.seq_le